## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.


1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) повторите пункт 4 на преобразованном корпусе;

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

In [2]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

## Данные

In [3]:
stops = set(stopwords.words('russian')) | {'gt',}
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def normalize(words):
    norm_words = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return norm_words

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [ ]:
# f = open('habr_texts.vw', 'w')

# for i, text in enumerate(habr_texts):
#     c = Counter(text)
#     doc = 'doc_'+ str(i) + ' '
#     vw_text = ' '.join([x+':'+str(c[x]) for x in c])
    
#     f.write(doc + vw_text  + '\n')
# f.close()



В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [4]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = [tokenize(remove_tags(text.lower())) for text in texts]

CPU times: user 3.04 s, sys: 452 ms, total: 3.49 s
Wall time: 3.66 s


In [5]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = [normalize(tokenize(text.lower())) for text in texts]

CPU times: user 13min 44s, sys: 5.21 s, total: 13min 50s
Wall time: 14min 2s


In [6]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

CPU times: user 10.7 s, sys: 592 ms, total: 11.2 s
Wall time: 11.4 s


In [ ]:
# для нграммов
# ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
# p = gensim.models.phrases.Phraser(ph)
# ngrammed_texts = p[texts]

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [19]:
def lda_gensim(texts, no_above=0.3, ngrams=True, tfidf=False, passes=10, eta='auto', iterations=10, num_topics=100):
    
    if ngrams:
        ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
        p = gensim.models.phrases.Phraser(ph)
        ngrammed_texts = p[texts]
        
    dictionary = gensim.corpora.Dictionary(texts)
    dictionary.filter_extremes(no_above=no_above)
    dictionary.compactify()
    
    corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]
    
    if tfidf:
        tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
        corpus = tfidf[corpus]
        
    lda_model = gensim.models.LdaModel(corpus=corpus, 
                                       id2word=dictionary,
                                       num_topics=num_topics,
                                       random_state=40,
                                       passes=passes,
                                       eta=eta, 
                                       iterations=iterations)
    return lda_model

In [20]:
%%time
lda_model= lda_gensim(texts)

CPU times: user 1min 17s, sys: 1.69 s, total: 1min 19s
Wall time: 1min 17s


In [21]:
lda_model.print_topics()

[(96,
  '0.017*"машина" + 0.012*"vpn" + 0.012*"to" + 0.011*"this" + 0.011*"выбор" + 0.009*"пост" + 0.009*"or" + 0.009*"т.е" + 0.009*"email" + 0.008*"вывод"'),
 (69,
  '0.018*"понять" + 0.017*"исследование" + 0.010*"тест" + 0.009*"почему" + 0.009*"продукт" + 0.009*"понимать" + 0.008*"какой-то" + 0.007*"тестирование" + 0.007*"цель" + 0.007*"этап"'),
 (2,
  '0.054*"сопротивляться" + 0.053*"чен" + 0.043*"всё-равно" + 0.021*"издательство" + 0.019*"вынуждать" + 0.019*"переименование" + 0.013*"продюсер" + 0.013*"хищник" + 0.010*"численность" + 0.010*"quest"'),
 (90,
  '0.030*"apple" + 0.026*"процессор" + 0.020*"железо" + 0.019*"память" + 0.018*"видеокарта" + 0.017*"чип" + 0.015*"производительность" + 0.015*"gpu" + 0.013*"ноутбук" + 0.012*"nvidia"'),
 (13,
  '0.023*"архитектура" + 0.014*"логика" + 0.013*"uber" + 0.013*"представление" + 0.012*"поток" + 0.011*"бизнес-логика" + 0.010*"состояние" + 0.010*"presenter" + 0.009*"содержимый" + 0.008*"элемент"'),
 (29,
  '0.032*"звук" + 0.028*"устройств

###### Постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем;

In [27]:
topics = [10, 25, 50, 100]

In [30]:
for i in topics:
    lda_model = lda_gensim(texts, passes=20, num_topics=i)
    print('%s:\n%s\n' % ('LDA %s' % i, lda_model.print_topics(10)))

LDA 10:
[(0, '0.047*"игра" + 0.019*"игрок" + 0.008*"играть" + 0.006*"понять" + 0.006*"игровой" + 0.006*"исследование" + 0.005*"машина" + 0.005*"тест" + 0.005*"карта" + 0.004*"понимать"'), (1, '0.009*"устройство" + 0.007*"сеть" + 0.006*"сайт" + 0.005*"цена" + 0.004*"смартфон" + 0.004*"интернет" + 0.003*"страна" + 0.003*"деньга" + 0.003*"товар" + 0.003*"производитель"'), (2, '0.014*"сервер" + 0.009*"сеть" + 0.009*"файл" + 0.006*"устройство" + 0.006*"настройка" + 0.005*"доступ" + 0.005*"пакет" + 0.004*"модуль" + 0.004*"запрос" + 0.004*"адрес"'), (3, '0.007*"элемент" + 0.006*"текст" + 0.006*"страница" + 0.006*"алгоритм" + 0.005*"язык" + 0.005*"изображение" + 0.005*"строка" + 0.004*"значение" + 0.004*"файл" + 0.004*"добавить"'), (4, '0.005*"мозг" + 0.005*"боль" + 0.004*"скорость" + 0.004*"учёный" + 0.004*"происходить" + 0.003*"технология" + 0.003*"исследование" + 0.003*"мир" + 0.003*"область" + 0.003*"сигнал"'), (5, '0.010*"модель" + 0.007*"звук" + 0.005*"инструмент" + 0.005*"компьютер" + 0

По моему мнению, три самые хорошие темы получились при num_topics = 100:

1. 
(33, '0.085*"видео" + 0.040*"регистратор" + 0.032*"корея" + 0.028*"модель" + 0.024*"запись" + 0.022*"blackvue" + 0.019*"камера" + 0.017*"аппарат" + 0.016*"gps" + 0.015*"китайский"')

2. 
(37, '0.025*"устройство" + 0.015*"камера" + 0.013*"смартфон" + 0.011*"датчик" + 0.010*"экран" + 0.010*"дом" + 0.006*"управление" + 0.006*"цена" + 0.005*"дисплей" + 0.005*"производитель"')

3. 
(94, '0.019*"сотрудник" + 0.017*"документ" + 0.010*"предприятие" + 0.010*"срок" + 0.010*"контроль" + 0.009*"управление" + 0.009*"оборудование" + 0.009*"внедрение" + 0.009*"руководство" + 0.008*"план"')


####  tfidf

In [34]:
for i in topics:
    lda_model = lda_gensim(texts, passes=20, num_topics=i, tfidf=True)
    print('%s:\n%s\n' % ('LDA %s' % i, lda_model.print_topics(10)))

LDA 10:
[(0, '0.001*"файл" + 0.001*"сервер" + 0.001*"устройство" + 0.001*"сеть" + 0.001*"сайт" + 0.001*"браузер" + 0.001*"игра" + 0.001*"объект" + 0.001*"программа" + 0.001*"метод"'), (1, '0.001*"файл" + 0.001*"сервер" + 0.001*"устройство" + 0.001*"yahoo" + 0.001*"сеть" + 0.001*"сайт" + 0.001*"браузер" + 0.001*"игра" + 0.001*"объект" + 0.001*"программа"'), (2, '0.001*"файл" + 0.001*"сервер" + 0.001*"устройство" + 0.001*"сеть" + 0.001*"сайт" + 0.001*"браузер" + 0.001*"игра" + 0.001*"объект" + 0.001*"программа" + 0.001*"метод"'), (3, '0.002*"файл" + 0.002*"безопасность" + 0.002*"сеть" + 0.001*"сервер" + 0.001*"браузер" + 0.001*"устройство" + 0.001*"windows" + 0.001*"сайт" + 0.001*"защита" + 0.001*"доступ"'), (4, '0.001*"файл" + 0.001*"сервер" + 0.001*"устройство" + 0.001*"сеть" + 0.001*"сайт" + 0.001*"браузер" + 0.001*"игра" + 0.001*"объект" + 0.001*"программа" + 0.001*"метод"'), (5, '0.002*"игра" + 0.001*"устройство" + 0.001*"сайт" + 0.001*"услуга" + 0.001*"продукт" + 0.001*"сеть" + 0.0

Темы стали хуже. В LDA 50 и 100 появилось много слов из программного кода и темы вообще неразборчивые.

Пример изменения темы в LDA 10 :

tfidf = False:

(2, '0.014*"сервер" + 0.009*"сеть" + 0.009*"файл" + 0.006*"устройство" + 0.006*"настройка" + 0.005*"доступ" + 0.005*"пакет" + 0.004*"модуль" + 0.004*"запрос" + 0.004*"адрес"')

tfidf = True:

(2, '0.001*"файл" + 0.001*"сервер" + 0.001*"устройство" + 0.001*"сеть" + 0.001*"сайт" + 0.001*"браузер" + 0.001*"игра" + 0.001*"объект" + 0.001*"программа" + 0.001*"метод"')

Пример изменения темы в LDA 25 :

tfidf = False:

(21, '0.016*"студент" + 0.014*"курс" + 0.009*"программа" + 0.008*"программирование" + 0.007*"участник" + 0.006*"обучение" + 0.006*"программист" + 0.005*"лекция" + 0.005*"знание" + 0.005*"электроника"')

tfidf = True:

(21, '0.004*"младенец" + 0.001*"мгту" + 0.001*"num" + 0.001*"фмрт" + 0.001*"чд" + 0.001*"svchost.exe" + 0.001*"файл" + 0.001*"устройство" + 0.001*"игра" + 0.001*"сервер"')

### Разложение матриц в sklearn

In [57]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [36]:
stexts = [' '.join(text) for text in texts]

In [109]:
def nmf_model(stexts, n_components=10, alpha=0, max_features=25000, min_df=5, max_df=0.3, lowercase=False, ngram_range=(1,2)):
        
    vectorizer = TfidfVectorizer(max_features=max_features,
                                 min_df=min_df,
                                 max_df=max_df,
                                 lowercase=lowercase)
    X = vectorizer.fit_transform(stexts)
    
    nmf_model = NMF(n_components=n_components, random_state=40, alpha=alpha)
    nmf_model.fit(X)
    
    return nmf_model, vectorizer

In [110]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(model.n_components):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [87]:
nmf_model, vectorizer = nmf_model(stexts)

In [88]:
get_nmf_topics(nmf_model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,книга,public,игра,космический,файл,клиент,lt,устройство,the,мозг
1,доклад,return,игрок,спутник,сервер,услуга,gt,камера,to,обучение
2,язык,void,игровой,орбита,запрос,сервис,div,смартфон,of,учёный
3,продукт,if,играть,земля,windows,рынок,react,звук,and,ия
4,программист,new,персонаж,вселенная,настройка,сеть,компонент,корпус,in,исследование
5,друг,string,unity,марс,страница,бизнес,return,телефон,is,алгоритм
6,программирование,var,steam,аппарат,строка,безопасность,const,цена,on,пациент
7,конференция,класс,vr,ракета,скрипт,инфраструктура,элемент,датчик,for,метод
8,опыт,private,движок,энергия,ключ,сайт,if,аккумулятор,end,клетка
9,заниматься,int,мобильный,планета,пакет,технология,int,процессор,if,нейросеть


##### Проделайте такие же действия для NMF

#### NMF 10

In [93]:
nmf_model, vectorizer = nmf_model(stexts, n_components=10, ngram_range=(1,2))

In [95]:
print('NMf 10:', get_nmf_topics(nmf_model, 10))

NMf 10:          Topic # 01 Topic # 02 Topic # 03   Topic # 04 Topic # 05  \
0             книга     public       игра  космический       файл   
1            доклад     return      игрок      спутник     сервер   
2              язык       void    игровой       орбита     запрос   
3           продукт         if     играть        земля    windows   
4       программист        new   персонаж    вселенная  настройка   
5              друг     string      unity         марс   страница   
6  программирование        var      steam      аппарат     строка   
7       конференция      класс         vr       ракета     скрипт   
8              опыт    private     движок      энергия       ключ   
9        заниматься        int  мобильный      планета      пакет   

       Topic # 06 Topic # 07   Topic # 08 Topic # 09    Topic # 10  
0          клиент         lt   устройство        the          мозг  
1          услуга         gt       камера         to      обучение  
2          сервис        

#### NMF 25

In [102]:
nmf_model, vectorizer = nmf_model(stexts, n_components=25, ngram_range=(1,2))

In [103]:
print('NMf 25:', get_nmf_topics(nmf_model, 10))

NMf 25:     Topic # 01 Topic # 02 Topic # 03   Topic # 04  Topic # 05  Topic # 06  \
0      продукт         if       игра  космический      объект       рынок   
1    сотрудник     return      игрок      спутник    значение       рубль   
2       клиент       else    игровой       орбита     элемент  российский   
3       бизнес        end     играть       ракета       метод       товар   
4         опыт        amp   персонаж         марс       класс      страна   
5     заказчик        var         vr      аппарат        блок      россия   
6  программист        int      steam        земля    алгоритм    блокчейн   
7       деньга   function      unity      станция       точка        цена   
8     менеджер       true     движок         наса    свойство      деньга   
9         идея      false   геймплей         луна  переменный     миллион   

  Topic # 07   Topic # 08 Topic # 09     Topic # 10     ...      Topic # 16  \
0         lt   устройство        the             ия     ...      

#### NMF 50

In [107]:
nmf_model, vectorizer = nmf_model(stexts, n_components=50, ngram_range=(1,2))

In [108]:
print('NMf 50:', get_nmf_topics(nmf_model, 10))

NMf 50:     Topic # 01      Topic # 02 Topic # 03   Topic # 04    Topic # 05  \
0    сотрудник             var       игра         марс    устройство   
1       деньга        function    игровой        земля     мобильный   
2  программист             new     играть  космический         умный   
3        жизнь          return      steam      аппарат        девайс   
4         опыт            case      unity      планета        гаджет   
5       понять             let     движок         луна  беспроводный   
6       думать           const   nintendo       орбита        датчик   
7         идея  xmlhttprequest       жанр      станция    управление   
8          кто             url         vr         наса           iot   
9       знание            true  приставка     астероид           дом   

   Topic # 06 Topic # 07   Topic # 08 Topic # 09     Topic # 10     ...       \
0       рубль         lt     смартфон        the             ия     ...        
1  российский         gt      телефон  

#### NMF 100

In [111]:
nmf_model, vectorizer = nmf_model(stexts, n_components=100, ngram_range=(1,2))

In [112]:
print('NMf 100:', get_nmf_topics(nmf_model, 10))

NMf 100:   Topic # 01 Topic # 02 Topic # 03 Topic # 04    Topic # 05       Topic # 06  \
0     деньга       true       игра     звезда    устройство       российский   
1      жизнь      false    игровой  галактика         умный            закон   
2     думать       func     играть   телескоп        девайс           россия   
3     понять       line      steam     солнце     мобильный           страна   
4     вообще       file      unity    планета        гаджет            рынок   
5   понимать       name     движок       небо        датчик               рф   
6       опыт       type   nintendo     светов  беспроводный         документ   
7        кто       null       жанр  солнечный    управление              сша   
8      ничто     string  приставка       свет          шлюз  государственный   
9       идея         ok   геймплей      земля           iot            китай   

  Topic # 07   Topic # 08 Topic # 09   Topic # 10      ...         Topic # 90  \
0         lt     смартфон    

##### В отдельной ячейке напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA

NMF в сравнении с LDA работает лучше, темы сформированы отлично. Однако, при 50 и 100 топиках снова появляется много мусора.
Самые хорошие получившиеся темы:
NMf 25:

Topic # 04

космический    
спутник       
орбита       
ракета      
марс         
аппарат 
земля   
станция
наса  
луна 

Topic # 17 
  
мозг    
учёный      
пациент    
клетка    
исследование      
заболевание      
болезнь    
ген  
врач     
организм 